In [1]:
import sys
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
sys.path.append("..")

# news scraping
from src.zinfo.news_scraper import NewsScraper

# clustering
from src.zinfo.article_clustering import get_vectorized_titles
from src.zinfo.article_clustering import get_num_clusters_per_val
from src.zinfo.article_clustering import cluster_articles

# article selecting
from src.zinfo.article_selector import get_best_article_all_clusters

In [50]:
def switch_columns(df, col1, col2):
    cols = list(df.columns)
    a, b = cols.index(col1), cols.index(col2)
    cols[b], cols[a] = cols[a], cols[b]
    dframe = df[cols]
    return dframe

In [2]:
start_date = "2021-08-13"
selected_news = pd.read_csv("../data/selected_articles.csv")
todays_articles = selected_news.loc[selected_news.date >= start_date]

In [3]:
todays_articles

,date,title,url,source,topic,num_articles
22,2021-08-13,How Marcus Maye treats Jets is example of why ...,https://nypost.com/2021/08/12/marcus-mayes-lea...,NaN,Free Guy,2
23,2021-08-13,‘The Premise’ Creator B.J. Novak On FX Antholo...,https://deadline.com/2021/08/the-premise-creat...,NaN,Ben Platt,453
24,2021-08-13,Virgil van Dijk: Liverpool defender signs deal...,https://www.bbc.co.uk/sport/football/58198972,NaN,Premier League,2
25,2021-08-13,Football without the fans: New study reveals e...,https://phys.org/news/2021-08-football-fans-re...,NaN,Premier League,3
26,2021-08-13,Acura Confirms Integra Return for 2022,https://www.thetruthaboutcars.com/2021/08/acur...,NaN,Acura Integra 2022,19
27,2021-08-13,"Nike Air Presto ""What The"" Calls Out a Plethor...",https://hypebeast.com/2021/8/nike-air-presto-w...,NaN,Travis Scott,2
28,2021-08-13,Kourtney Kardashian Responds to Body Shamers W...,https://www.complex.com/pop-culture/kourtney-k...,NaN,Travis Scott,5
29,2021-08-13,Kourtney Kardashian goes MAKE-UP FREE and wear...,https://www.dailymail.co.uk/tvshowbiz/article-...,NaN,Travis Scott,2
30,2021-08-13,November 2021 DC Comics revealed: A whole lot ...,https://www.gamesradar.com/dc-solicitations-no...,NaN,Travis Scott,3
31,2021-08-13,"Everton news: James - Benitez feud latest, Ins...",https://royalbluemersey.sbnation.com/2021/8/13...,NaN,La Liga,3
